In [66]:
!pip install pprint

ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [1]:
from pprint import pprint
from lxml import html
import requests
import time
from pymongo import MongoClient

In [2]:
header = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
}

In [3]:

# Функция для получения DOM сайта. В качестве аргумента принимает ссылку
def get_dom(link):
    header = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    response = requests.get(link, headers=header)
    dom = html.fromstring(response.text)
    return dom

# Функция для очистки списка и отображения первого элемента. В качестве аргумента принимает список
def list_len(list):
    try:
        el = list[0].replace('\xa0', ' ')
        return el
    except:
        pass
# Функция конструктор ссылок. В качестве аргумента принимает основную ссылку и сокращенную    
def link_constr(main_link, link):
    if 'https://' not in link:
        link = main_link + link
    else:
        link
    return link

# Функция добавления документов в коллекции. В качестве аргументов принимает добавляемый словарь и коллекцию для добавления
def add_collection(news_dict, collection):
    for new in news_dict:
        count = 0
        if collection.count_documents({'title': new['title']}) == 0:
            collection.insert_one(new)
            count += 1
    print(f'В коллекцию {collection.name} добавлено {count} документов. Всего документов в коллекции {collection.count_documents({})}')

#### mail

In [4]:
mail_url = 'https://news.mail.ru'
dom_mail = get_dom(mail_url)

response = requests.get(mail_url, headers = header)   
response

<Response [200]>

In [5]:
section = dom_mail.xpath("//div[contains(@data-counter-id, '20268335')]//a[contains(@class, photo)]")

mail_news =[]

for new in section:
    mail_new = {}

    link = list_len(new.xpath(".//@href"))
    link = link_constr(mail_url, link)
    
    temp_dom = get_dom(link)
    
    title = list_len(temp_dom.xpath("//h1/text()"))
    date = list_len(temp_dom.xpath("//span[@class = 'note__text breadcrumbs__text js-ago']/@datetime"))
    source = list_len(temp_dom.xpath("//a[@class = 'link color_gray breadcrumbs__link']//span/text()"))
    
    mail_new['source'] = source
    mail_new['title'] = title
    mail_new['link'] = link
    mail_new['date'] = date   
    
    mail_news.append(mail_new)
    
pprint(mail_news[0])

{'date': '2023-01-12T07:42:27+03:00',
 'link': 'https://news.mail.ru/politics/54622984/',
 'source': '© РИА Новости',
 'title': 'Постпредство России анонсировало заседание СБ ООН по Украине'}


In [6]:
client = MongoClient('localhost', 27017)
db = client['news_database']
news_mail = db.news_mail


In [7]:
add_collection(mail_news, news_mail)

В коллекцию news_mail добавлено 0 документов. Всего документов в коллекции 11
